In [0]:
!pip install kaggle

    100% |████████████████████████████████| 51kB 2.5MB/s 
    100% |████████████████████████████████| 51kB 5.9MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/3b/8e/14/adcbb71d126f5fb939fec8aa81394450922c20eee876bf8882
Successfully built kaggle


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

In [0]:
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!kaggle datasets download -d tohidul/food11

100%|██████████████████████████████████████▉| 1.37G/1.37G [00:15<00:00, 107MB/s]
100%|██████████████████████████████████████| 1.37G/1.37G [00:15<00:00, 92.5MB/s]


In [0]:
import zipfile

In [0]:
zf = zipfile.ZipFile('/content/.kaggle/datasets/tohidul/food11/food11re.zip', "r")
zf.extractall()

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

Using TensorFlow backend.


In [0]:
from keras import optimizers

In [0]:
from keras.optimizers import Adam

In [0]:
train_data_dir = "food11re/training"
val_data_dir = "food11re/validation"
batch_size = 64
n_classes = 11
data_augmentor = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False, # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')
target_size = (224, 224)
target_dims = (224, 224, 3)

In [0]:
train_generator = data_augmentor.flow_from_directory(train_data_dir, target_size=target_size, batch_size=batch_size, shuffle=True)

Found 9866 images belonging to 11 classes.


In [0]:
val_generator = data_augmentor.flow_from_directory(val_data_dir, target_size=target_size, batch_size=batch_size)

Found 3430 images belonging to 11 classes.


In [0]:
nb_classes = 11

In [0]:
model = Sequential()
model.add(Convolution2D(32, (3,3), padding='same', input_shape=target_dims))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Convolution2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Convolution2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(11))
model.add(Activation("softmax"))
    

In [0]:
epochs = 100

In [0]:
opt = Adam(lr=1e-3, decay=1e-3 / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [0]:
nb_epochs = 30

In [0]:
import keras

In [0]:

opt = SGD(lr=.01, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
sgd = optimizers.SGD(lr=0.3, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['acc'])

In [0]:
print(train_generator.n/train_generator.batch_size)

154.15625


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [0]:
STEP_SIZE_TRAIN=64
STEP_SIZE_VALID=64

In [0]:
model.fit_generator(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_steps=STEP_SIZE_VALID,      
          validation_data=(val_generator),
                    verbose = 1,
                   epochs=epochs)

Epoch 1/100
154/154 [==============================] - 265s 2s/step - loss: 2.7169 - acc: 0.2142 - val_loss: 2.6811 - val_acc: 0.2580
Epoch 2/100
154/154 [==============================] - 267s 2s/step - loss: 2.5108 - acc: 0.2518 - val_loss: 2.4135 - val_acc: 0.2538
Epoch 3/100
139/154 [==========================>...] - ETA: 19s - loss: 2.3270 - acc: 0.2889

154/154 [==============================] - 267s 2s/step - loss: 2.3187 - acc: 0.2881 - val_loss: 2.4032 - val_acc: 0.3101
Epoch 4/100
154/154 [==============================] - 275s 2s/step - loss: 2.1824 - acc: 0.3071 - val_loss: 2.2179 - val_acc: 0.2736
Epoch 5/100
154/154 [==============================] - 275s 2s/step - loss: 2.0396 - acc: 0.3456 - val_loss: 2.5067 - val_acc: 0.2624
Epoch 6/100
 22/154 [===>..........................] - ETA: 2:25 - loss: 1.9712 - acc: 0.3665

154/154 [==============================] - 268s 2s/step - loss: 1.9834 - acc: 0.3627 - val_loss: 1.9386 - val_acc: 0.3520
Epoch 7/100
154/154 [==============================] - 272s 2s/step - loss: 1.8533 - acc: 0.3694 - val_loss: 1.7637 - val_acc: 0.3989
Epoch 8/100
154/154 [==============================] - 277s 2s/step - loss: 1.7527 - acc: 0.4005 - val_loss: 5.2780 - val_acc: 0.1784
Epoch 9/100
  3/154 [..............................] - ETA: 57s - loss: 1.7412 - acc: 0.3438

154/154 [==============================] - 270s 2s/step - loss: 1.7428 - acc: 0.4136 - val_loss: 4.4441 - val_acc: 0.1639
Epoch 10/100
154/154 [==============================] - 270s 2s/step - loss: 1.7371 - acc: 0.4017 - val_loss: 1.6986 - val_acc: 0.4180
Epoch 11/100
154/154 [==============================] - 275s 2s/step - loss: 1.6246 - acc: 0.4427 - val_loss: 3.7215 - val_acc: 0.1524


Epoch 12/100
154/154 [==============================] - 277s 2s/step - loss: 1.6382 - acc: 0.4406 - val_loss: 1.5192 - val_acc: 0.4932
Epoch 13/100
154/154 [==============================] - 271s 2s/step - loss: 1.5084 - acc: 0.4868 - val_loss: 2.5842 - val_acc: 0.2925
Epoch 14/100
137/154 [=========================>....] - ETA: 20s - loss: 1.4637 - acc: 0.5000

...

154/154 [==============================] - 265s 2s/step - loss: 0.4992 - acc: 0.8320 - val_loss: 1.0412 - val_acc: 0.6955
Epoch 91/100
154/154 [==============================] - 270s 2s/step - loss: 0.5205 - acc: 0.8207 - val_loss: 1.2459 - val_acc: 0.6583
Epoch 92/100
153/154 [============================>.] - ETA: 1s - loss: 0.5995 - acc: 0.7972

154/154 [==============================] - 264s 2s/step - loss: 0.5984 - acc: 0.7978 - val_loss: 1.1725 - val_acc: 0.6601
Epoch 93/100
154/154 [==============================] - 270s 2s/step - loss: 0.4909 - acc: 0.8303 - val_loss: 0.8736 - val_acc: 0.7409
Epoch 94/100
124/154 [=======================>......] - ETA: 36s - loss: 0.5694 - acc: 0.8105

154/154 [==============================] - 269s 2s/step - loss: 0.5521 - acc: 0.8144 - val_loss: 0.8727 - val_acc: 0.7388
Epoch 95/100
154/154 [==============================] - 268s 2s/step - loss: 0.5307 - acc: 0.8239 - val_loss: 1.1954 - val_acc: 0.6654
Epoch 96/100
154/154 [==============================] - 270s 2s/step - loss: 0.4668 - acc: 0.8466 - val_loss: 0.9634 - val_acc: 0.7155
Epoch 97/100
 19/154 [==>...........................] - ETA: 1:28 - loss: 0.4533 - acc: 0.8470

154/154 [==============================] - 265s 2s/step - loss: 0.4921 - acc: 0.8297 - val_loss: 1.5124 - val_acc: 0.6362
Epoch 98/100
154/154 [==============================] - 265s 2s/step - loss: 0.4823 - acc: 0.8319 - val_loss: 0.8942 - val_acc: 0.7426
Epoch 99/100
154/154 [==============================] - 271s 2s/step - loss: 0.4654 - acc: 0.8402 - val_loss: 1.0037 - val_acc: 0.7185
Epoch 100/100
  1/154 [..............................] - ETA: 1:00 - loss: 0.4879 - acc: 0.8125

154/154 [==============================] - 268s 2s/step - loss: 0.4552 - acc: 0.8463 - val_loss: 0.9684 - val_acc: 0.7226
